In [ ]:
import numpy as np

#from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.tri as tri

from astropy import units as u
from astropy.io import ascii

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
ACT = 104
NODE = 3222
BRAD = 3228.5

In [ ]:
NODE*ACT

In [ ]:
# for each node, the influence from each actuator
surf2act = np.fromfile("../mmtwfs/data/Surf2ActTEL_32.bin", dtype=np.float32).reshape(ACT, NODE).T
nodecoor = ascii.read("../mmtwfs/data/bcv_node_coordinates.dat", names=["bcv_id", "bcv_x", "bcv_y", "bcv_z"])
actcoor = ascii.read("../mmtwfs/data/actuator_coordinates.dat", names=["act_i", "act_x", "act_y", "act_type"])
actcoor.add_index("act_i")
for ax in ["bcv_x", "bcv_y"]:
    nodecoor[ax] /= BRAD
nodecoor['bcv_rho'] = np.sqrt(nodecoor['bcv_x']**2 + nodecoor['bcv_y']**2)
nodecoor['bcv_phi'] = np.arctan2(nodecoor['bcv_y'], nodecoor['bcv_x'])

In [ ]:
np.min(nodecoor['bcv_rho']), np.max(nodecoor['bcv_rho'])

In [ ]:
surf2act

In [ ]:
surf2act_full = np.loadtxt("../mmtwfs/data/Surf2ActTEL_104").T
act2surf = np.loadtxt("../mmtwfs/data/Act2SurfTEL").T
act2surf, surf2act_full

In [ ]:
np.ones(ACT) @ act2surf

In [ ]:
surf2act_full / surf2act

In [ ]:
def plot_actuator_influence(act_id):
    actuator_info = actcoor.loc[act_id]
    actuator = actuator_info['act_i']
    act_index = actcoor['act_i'].searchsorted(actuator)
    fig, ax = plt.subplots()
    fig.set_label(f"Actuator ID: {actuator}")
    fvec = np.zeros(ACT)
    fvec[act_index] = 1.0
    ph = fvec @ act2surf
    X, Y = nodecoor['bcv_x'].value, nodecoor['bcv_y'].value
    triang = tri.Triangulation(X, Y)
    # Mask off hole in center of mirror
    triang.set_mask(
        np.hypot(
            X[triang.triangles].mean(axis=1),
            Y[triang.triangles].mean(axis=1)
        ) < np.min(nodecoor['bcv_rho'])
    )
    ax.set_aspect('equal')
    vmax = np.ceil(np.max(np.abs(ph))+1)
    vmin = -vmax+1
    levels = np.arange(vmin, vmax)
    tcf = ax.tricontourf(triang, ph, vmax=vmax, vmin=vmin, levels=levels, cmap=cm.RdBu)
    ax.set_axis_off()
    fig.colorbar(tcf, ax=ax, label='Wavefront Error (nm)')
    ax.tricontour(triang, ph, vmax=vmax, vmin=vmin, levels=levels, colors='k')
    xcor = actcoor['act_x']/BRAD
    ycor = actcoor['act_y']/BRAD
    ax.scatter(xcor, ycor, marker='.', color='black')
    for i, (x, y) in enumerate(zip(xcor, ycor)):
        ax.text(
            x,
            y + 0.02,
            actcoor["act_i"][i],
            horizontalalignment="center",
            verticalalignment="bottom",
            size="xx-small",
            color="black"
        )
    ax.scatter(actuator_info['act_x']/BRAD, actuator_info['act_y']/BRAD, color='lime', marker='*')
    return fig

In [ ]:
fig = plot_actuator_influence(115)


In [ ]:
actcoor.loc[8]